# 1. Model training

In [1]:
import numpy as np
import pandas as pd
import warnings
import os
import pickle
import optuna

import sklearn
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.datetime import DatetimeFeatures
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from feature_engine.encoding import RareLabelEncoder, MeanEncoder, CountFrequencyEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import r2_score

# Display settings

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
sklearn.set_config(transform_output='pandas')

In [4]:
warnings.filterwarnings("ignore")

# 2. Read the data

In [5]:
file_path = r"C:\Users\User\OneDrive\Desktop\Flight-Price-Prediction\Data"

In [6]:
train = pd.read_csv(f"{file_path}\\train.csv")

In [7]:
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087
...,...,...,...,...,...,...,...,...,...,...
6690,Jet Airways,2019-03-21,Delhi,Cochin,10:45:00,18:50:00,1925,2.0,No Info,11093
6691,Air India,2019-05-01,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8891
6692,Jet Airways,2019-06-01,Delhi,Cochin,14:00:00,19:00:00,300,1.0,In-flight meal not included,10262
6693,Air Asia,2019-06-24,Delhi,Cochin,07:55:00,13:25:00,330,1.0,No Info,6152


In [8]:
val = pd.read_csv(f"{file_path}\\val.csv")

In [9]:
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704
...,...,...,...,...,...,...,...,...,...,...
1669,Spicejet,2019-05-01,Chennai,Kolkata,09:45:00,12:00:00,135,0.0,No Info,3597
1670,Indigo,2019-05-01,Kolkata,Banglore,08:10:00,13:00:00,290,1.0,No Info,5069
1671,Jet Airways,2019-05-27,Delhi,Cochin,05:30:00,12:35:00,425,2.0,In-flight meal not included,15544
1672,Jet Airways,2019-06-12,Mumbai,Hyderabad,19:35:00,21:05:00,90,0.0,In-flight meal not included,3210


In [10]:
test = pd.read_csv(f"{file_path}\\test.csv")

In [11]:
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804
...,...,...,...,...,...,...,...,...,...,...
2088,Jet Airways,2019-05-27,Delhi,Cochin,19:15:00,12:35:00,1040,1.0,In-flight meal not included,12898
2089,Multiple Carriers,2019-06-27,Delhi,Cochin,11:25:00,19:15:00,470,1.0,No Info,7155
2090,Jet Airways,2019-06-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,In-flight meal not included,11627
2091,Multiple Carriers,2019-06-06,Delhi,Cochin,15:15:00,01:30:00,615,1.0,No Info,6795


# 3. Preprocessing operations

In [12]:
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("grouper", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

In [13]:
feature_to_extract = ["month", "week", "day_of_week", "day_of_month", "weekend", "year_start", "year_end"]

In [14]:
doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scaler", MinMaxScaler())
])

In [15]:
location_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="other", n_categories=2)),
    ("encoder", MeanEncoder()),
    ("scaler", PowerTransformer())
])

In [16]:
def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Mumbai", "New Delhi"]
    
    return (
        X
        .assign(**{
            f"{col}_is_north" : X.loc[:, col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

In [17]:
loc_transformer = Pipeline(steps=[
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func=is_north))
])

In [18]:
time_pipe1 = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
    ("scaler", MinMaxScaler())
])

In [19]:
def part_of_day(X, morning=4, afternoon=12, evening=16, night=20):
    columns = X.columns.to_list()
    
    X_temp = X.assign(**{
        col : pd.to_datetime(X.loc[:, col]).dt.hour
        for col in columns
    })
    
    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day" : np.select(
                [X_temp.loc[:, col].between(morning, afternoon, inclusive="left"),
                X_temp.loc[:, col].between(afternoon, evening, inclusive="left"),
                X_temp.loc[:, col].between(evening, night, inclusive="left")],
                ["morning", "afternoon", "evening"],
                default="night"
            )
            for col in columns
        })
        .drop(columns=columns)
    )

In [20]:
time_pipe2 = Pipeline(steps=[
    ("funcT", FunctionTransformer(func=part_of_day)),
    ("encoder", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

In [21]:
time_transformer = FeatureUnion(transformer_list=[
    ("part1", time_pipe1),
    ("part2", time_pipe2)
])

In [22]:
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma
        
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        if not self.variables:
            self.variables = X.select_dtypes(include="number").columns.to_list()
            
        self.reference_values_ = {
            col : (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1, 1)
            )
            for col in self.variables
        }
    
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile*100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data = rbf_kernel(X.loc[:, [col]], Y = self.reference_values_[col] , gamma = self.gamma),
                columns=columns
            )
                
            objects.append(obj)
                
        return pd.concat(objects, axis=1)

In [23]:
def classify_duration(X, short=180, medium=400):
    return (
        X
        .assign(duration_modified = np.select(
                [X.duration.lt(short),
                X.duration.between(short, medium, inclusive="left")],
                ["short", "medium"],
                default="long"
            ))
        .drop(columns="duration")
    )

In [24]:
def is_over(X, value=1000):
    return (
        X
        .assign(**{
            f"duration_over{value}":X.duration.gt(value).astype(int)
        })
        .drop(columns="duration")
    )

In [25]:
duration_pipe1 = Pipeline(steps=[
    ("rbf", RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
    ("cat", FunctionTransformer(func=classify_duration)),
    ("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func=is_over)),
    ("part4", StandardScaler())
]) 

duration_transformer = Pipeline(steps=[
    ("outliers", Winsorizer(capping_method='iqr', fold=1.5)),
    ("imputer", SimpleImputer(strategy="median")),
    ("union", duration_union)
])

In [26]:
def is_direct(X):
    return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))

In [27]:
total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("", FunctionTransformer(func=is_direct))
])

In [28]:
info_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

In [29]:
def have_info(X):
    return X.assign(additional_info=X.additional_info.ne("No info").astype(int))

In [30]:
info_union = FeatureUnion(transformer_list=[
    ("part1", info_pipe1),
    ("part2", FunctionTransformer(func=have_info))
])

In [31]:
info_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ("union", info_union)
])

In [32]:
col_transformer = ColumnTransformer(transformers=[
    ("t1", air_transformer, ["airline"]),
    ("t2", doj_transformer, ["date_of_journey"]),
    ("t3", loc_transformer, ["source", "destination"]),
    ("t4", time_transformer, ["arrival_time", "dep_time"]),
    ("t5", duration_transformer, ["duration"]),
    ("t6", total_stops_transformer, ["total_stops"]),
    ("t7", info_transformer, ["additional_info"])
], remainder="passthrough")

In [33]:
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
    estimator=estimator,
    scoring="r2",
    threshold=0.1
)

In [34]:
preprocessor = Pipeline(steps=[
    ("ct", col_transformer),
    ("selector", selector)
])

In [35]:
preprocessor.fit(train.drop(columns="price"),
                train.price.copy())

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t1',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('t2',
                                                  Pipeline(steps=[('dt',
                                                                   DatetimeFe...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x00000228835402C0>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [36]:
train_transformed = preprocessor.transform(train.drop(columns="price"))

In [37]:
train_transformed

,t1__airline_Indigo,t1__airline_Jet Airways,t1__airline_Other,t2__date_of_journey_week,t5__duration_rbf_25,t5__duration_modified,t5__duration_over1000,t5__duration,t6__total_stops,t6__is_direct_flight
0,0.0,1.0,0.0,0.176471,-0.364262,2.0,0,-0.033916,1.0,0
1,0.0,1.0,0.0,0.235294,-0.364262,2.0,0,0.046422,1.0,0
2,0.0,0.0,1.0,0.058824,2.373008,0.0,0,-0.917631,0.0,1
3,0.0,0.0,0.0,0.882353,-0.364262,2.0,0,-0.174507,1.0,0
4,0.0,1.0,0.0,0.117647,-0.364262,2.0,0,-0.214676,1.0,0
...,...,...,...,...,...,...,...,...,...,...
6690,0.0,1.0,0.0,0.176471,-0.364262,2.0,1,2.597145,2.0,0
6691,0.0,0.0,0.0,0.529412,-0.364262,2.0,0,-0.174507,1.0,0
6692,0.0,1.0,0.0,0.764706,-0.364262,1.0,0,-0.666576,1.0,0
6693,0.0,0.0,1.0,1.000000,-0.364262,1.0,0,-0.606322,1.0,0


In [38]:
val_transformed = preprocessor.transform(val.drop(columns="price"))

In [39]:
val_transformed

,t1__airline_Indigo,t1__airline_Jet Airways,t1__airline_Other,t2__date_of_journey_week,t5__duration_rbf_25,t5__duration_modified,t5__duration_over1000,t5__duration,t6__total_stops,t6__is_direct_flight
0,1.0,0.0,0.0,1.000000,-0.364262,1.0,0,-0.656533,1.0,0
1,0.0,0.0,0.0,0.882353,-0.364262,2.0,0,0.267351,1.0,0
2,0.0,1.0,0.0,0.117647,-0.364262,2.0,0,0.699166,1.0,0
3,0.0,0.0,0.0,0.823529,-0.364262,2.0,0,-0.305056,1.0,0
4,0.0,1.0,0.0,0.647059,-0.364262,2.0,1,2.265752,2.0,0
...,...,...,...,...,...,...,...,...,...,...
1669,0.0,0.0,1.0,0.529412,-0.364262,0.0,0,-0.997969,0.0,1
1670,1.0,0.0,0.0,0.529412,-0.364262,1.0,0,-0.686660,1.0,0
1671,0.0,1.0,0.0,0.764706,-0.364262,2.0,0,-0.415520,2.0,0
1672,0.0,1.0,0.0,0.882353,-0.364262,0.0,0,-1.088349,0.0,1


In [40]:
test_transformed = preprocessor.transform(test.drop(columns="price"))

In [41]:
test_transformed

,t1__airline_Indigo,t1__airline_Jet Airways,t1__airline_Other,t2__date_of_journey_week,t5__duration_rbf_25,t5__duration_modified,t5__duration_over1000,t5__duration,t6__total_stops,t6__is_direct_flight
0,0.0,1.0,0.0,0.058824,-0.364262,2.0,1,1.653176,1.0,0
1,0.0,0.0,1.0,0.823529,-0.364262,0.0,0,-0.987926,0.0,1
2,1.0,0.0,0.0,0.176471,3.112489,0.0,0,-0.927673,0.0,1
3,1.0,0.0,0.0,1.000000,-0.364262,0.0,0,-0.987926,0.0,1
4,1.0,0.0,0.0,0.588235,-0.364262,0.0,0,-0.967842,0.0,1
...,...,...,...,...,...,...,...,...,...,...
2088,0.0,1.0,0.0,0.764706,-0.364262,2.0,1,0.819672,1.0,0
2089,0.0,0.0,0.0,1.000000,-0.364262,2.0,0,-0.325140,1.0,0
2090,0.0,1.0,0.0,0.823529,-0.364262,2.0,1,1.884147,1.0,0
2091,0.0,0.0,0.0,0.823529,-0.364262,2.0,0,-0.033916,1.0,0


# 4. Training the model

In [42]:
def objective(trial):
    dtrain = xgb.DMatrix(train_transformed, label=train["price"])
    dval = xgb.DMatrix(val_transformed, label=val["price"])
    
    param = {
        'objective': 'reg:squarederror',
        'eta': trial.suggest_float('eta', 0.05, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'alpha': trial.suggest_float('alpha', 0, 1)
    }
    
    flight = xgb.train(param, dtrain, 
                   num_boost_round=100, evals=[(dval, 'validation')], 
                   early_stopping_rounds=10, 
                   maximize=True)
    
    # Predict on the validation set
    preds = flight.predict(dval)
    r2 = r2_score(val["price"], preds)
    
    return r2

In [43]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-06-17 19:54:57,809] A new study created in memory with name: no-name-290be5b0-d0b2-46e3-bf58-3fcc6f639276


[0]	validation-rmse:4443.45183
[1]	validation-rmse:4208.99773
[2]	validation-rmse:3928.20282
[3]	validation-rmse:3714.90409
[4]	validation-rmse:3519.39349
[5]	validation-rmse:3340.24398
[6]	validation-rmse:3200.67875
[7]	validation-rmse:3103.16018
[8]	validation-rmse:3026.95378
[9]	validation-rmse:2955.81746


[I 2024-06-17 19:54:57,947] Trial 0 finished with value: 0.6210654377937317 and parameters: {'eta': 0.11869271789346093, 'max_depth': 5, 'alpha': 0.15100855323559936}. Best is trial 0 with value: 0.6210654377937317.


[0]	validation-rmse:4462.58231
[1]	validation-rmse:4247.50798
[2]	validation-rmse:4021.77440
[3]	validation-rmse:3836.85117
[4]	validation-rmse:3683.84019
[5]	validation-rmse:3559.86158
[6]	validation-rmse:3453.79250
[7]	validation-rmse:3361.75251
[8]	validation-rmse:3286.91893
[9]	validation-rmse:3223.06447


[I 2024-06-17 19:54:58,067] Trial 1 finished with value: 0.5495844483375549 and parameters: {'eta': 0.1176441131331355, 'max_depth': 3, 'alpha': 0.9203525308304396}. Best is trial 0 with value: 0.6210654377937317.


...

[I 2024-06-17 19:55:04,830] Trial 47 finished with value: 0.5851845741271973 and parameters: {'eta': 0.09905708740515076, 'max_depth': 5, 'alpha': 0.39908290711830285}. Best is trial 42 with value: 0.679989218711853.


[0]	validation-rmse:4282.74080
[1]	validation-rmse:3961.32231
[2]	validation-rmse:3610.96634
[3]	validation-rmse:3371.20660
[4]	validation-rmse:3185.15172
[5]	validation-rmse:3049.72481
[6]	validation-rmse:2957.25126
[7]	validation-rmse:2888.13914
[8]	validation-rmse:2841.01850
[9]	validation-rmse:2784.00574
[10]	validation-rmse:2760.64774


[I 2024-06-17 19:55:04,997] Trial 48 finished with value: 0.6576789617538452 and parameters: {'eta': 0.19611990054305042, 'max_depth': 4, 'alpha': 0.3258454979250355}. Best is trial 42 with value: 0.679989218711853.


[0]	validation-rmse:4311.32696
[1]	validation-rmse:4000.98025
[2]	validation-rmse:3637.41518
[3]	validation-rmse:3390.46924
[4]	validation-rmse:3185.65378
[5]	validation-rmse:3012.03251
[6]	validation-rmse:2898.33303
[7]	validation-rmse:2828.78902
[8]	validation-rmse:2779.70112
[9]	validation-rmse:2741.31259


[I 2024-06-17 19:55:05,181] Trial 49 finished with value: 0.6667884588241577 and parameters: {'eta': 0.17847880001982502, 'max_depth': 5, 'alpha': 0.6707173395358084}. Best is trial 42 with value: 0.679989218711853.


In [44]:
# Get the best hyperparameters
print('Best hyperparameters: ', study.best_params)

Best hyperparameters:  {'eta': 0.1998750163296871, 'max_depth': 5, 'alpha': 0.4386990351118196}


In [47]:
# Train the model with the best hyperparameters on the full training set
best_params = study.best_params
best_params['objective'] = 'reg:squarederror'
dtrain = xgb.DMatrix(train_transformed, label=train["price"])
dtest = xgb.DMatrix(test_transformed, label=test["price"])
flight = xgb.train(best_params, dtrain, num_boost_round=100)

In [52]:
with open('xgboost-best-model.pkl', 'wb') as file:
    pickle.dump(flight, file)

In [53]:
y_pred = flight.predict(dtest)

In [54]:
r2 = r2_score(test["price"], y_pred)
print(f'Test R2: {r2:.2f}')

Test R2: 0.73
